# 🎓 The Virtual NMR Spectrometer

Welcome to the **Interactive Relaxation Lab**! 

This notebook simulates a **Nuclear Magnetic Resonance (NMR)** experiment on a virtual protein.
NMR relaxation rates ($R_1$, $R_2$, and Heteronuclear NOE) are the primary tools biophysicists use to study protein movement at the atomic level.

### 🎯 Goal
Understand how **molecular size** (tumbling rate) and **magnetic field strength** affect the signals we measure. This simulation uses the standard *Lipari-Szabo Model-Free* formalism, which is the gold standard for analyzing protein dynamics.

In [ ]:
# ⚙️ Install Dependencies
# We install openmm via pip for compatibility.
# We force reinstall synth-pdb from GitHub to avoid caching old versions.
try:
    import google.colab
    print("Installing OpenMM and synth-pdb...")
    !pip install -q openmm matplotlib ipywidgets py3Dmol
    !pip install -q --force-reinstall --no-cache-dir git+https://github.com/elkins/synth-pdb.git@master
except ImportError:
    print("Assuming local environment.")
    pass


In [ ]:
# ⚙️ HOTFIX: Patch synth_pdb for Colab compatibility (Physics + Generator + J-Coupling)
# This cell ensures the code handles missing OpenMM gracefully, preserves B-factors, AND adds J-Coupling physics.
import os
import synth_pdb
import base64
import importlib

package_dir = os.path.dirname(synth_pdb.__file__)
physics_file = os.path.join(package_dir, "physics.py")
generator_file = os.path.join(package_dir, "generator.py")
jc_file = os.path.join(package_dir, "j_coupling.py")

print(f"🚨 Applying robust hotfix to: {physics_file}, {generator_file}, {jc_file}")

encoded_physics = "aW1wb3J0IGxvZ2dpbmcKdHJ5OgogICAgaW1wb3J0IG9wZW5tbS5hcHAgYXMgYXBwCiAgICBpbXBvcnQgb3Blbm1tIGFzIG1tCiAgICBmcm9tIG9wZW5tbSBpbXBvcnQgdW5pdAogICAgSEFTX09QRU5NTSA9IFRydWUKZXhjZXB0IEltcG9ydEVycm9yOgogICAgSEFTX09QRU5NTSA9IEZhbHNlCiAgICBhcHAgPSBOb25lCiAgICBtbSA9IE5vbmUKICAgIHVuaXQgPSBOb25lCmltcG9ydCBzeXMKaW1wb3J0IG9zCgpsb2dnZXIgPSBsb2dnaW5nLmdldExvZ2dlcihfX25hbWVfXykKCmNsYXNzIEVuZXJneU1pbmltaXplcjoKICAgICIiIgogICAgUGVyZm9ybXMgZW5lcmd5IG1pbmltaXphdGlvbiBvbiBtb2xlY3VsYXIgc3RydWN0dXJlcyB1c2luZyBPcGVuTU0uCiAgICAKICAgICMjIyBFZHVjYXRpb25hbCBOb3RlOiBXaGF0IGlzIEVuZXJneSBNaW5pbWl6YXRpb24/CiAgICAtLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLQogICAgUHJvdGVpbnMgZm9sZCBpbnRvIHNwZWNpZmljIDNEIHNoYXBlcyB0byBtaW5pbWl6ZSB0aGVpciAiR2liYnMgRnJlZSBFbmVyZ3kiLgogICAgQSBnZW5lcmF0ZWQgc3RydWN0dXJlIChsaWtlIG9uZSBidWlsdCBmcm9tIHNpbXBsZSBnZW9tZXRyeSkgb2Z0ZW4gaGFzICJjbGFzaGVzIgogICAgd2hlcmUgYXRvbXMgYXJlIHRvbyBjbG9zZSAoaGlnaCBWYW4gZGVyIFdhYWxzIHJlcHVsc2lvbikgb3IgYm9uZCBhbmdsZXMgYXJlIHN0cmFpbmVkLgogICAgCiAgICBFbmVyZ3kgTWluaW1pemF0aW9uIGlzIGxpa2Ugcm9sbGluZyBhIGJhbGwgZG93biBhIGhpbGwuIFRoZSAiRW5lcmd5IExhbmRzY2FwZSIKICAgIHJlcHJlc2VudHMgdGhlIHBvdGVudGlhbCBlbmVyZ3kgb2YgdGhlIHByb3RlaW4gYXMgYSBmdW5jdGlvbiBvZiBhbGwgaXRzIGF0b20gY29vcmRpbmF0ZXMuCiAgICBUaGUgYWxnb3JpdGhtIG1vdmVzIGF0b21zIHNsaWdodGx5IHRvIHJlZHVjZSB0aGlzIGVuZXJneSwgZmluZGluZyBhIGxvY2FsIG1pbmltdW0KICAgIHdoZXJlIHRoZSBzdHJ1Y3R1cmUgaXMgcGh5c2ljYWxseSByZWxheGVkLgoKICAgICMjIyBOTVIgUGVyc3BlY3RpdmU6CiAgICBJbiBOTVIgc3RydWN0dXJlIGNhbGN1bGF0aW9uIChlLmcuLCBDWUFOQSwgWFBMT1ItTklIKSwgbWluaW1pemF0aW9uIGlzIG9mdGVuIHBhcnQgb2YKICAgICJTaW11bGF0ZWQgQW5uZWFsaW5nIi4gU3RydWN0dXJlcyBhcmUgY2FsY3VsYXRlZCB0byBzYXRpc2Z5IGV4cGVyaW1lbnRhbCByZXN0cmFpbnRzCiAgICAoTk9FcywgSi1jb3VwbGluZ3MpIGFuZCB0aGVuIGVuZXJneS1taW5pbWl6ZWQgdG8gZW5zdXJlIGdvb2QgZ2VvbWV0cnkuCiAgICBUaGlzIG1vZHVsZSBwZXJmb3JtcyB0aGF0IGZpbmFsICJnZW9tZXRyeSByZWd1bGFyaXphdGlvbiIgc3RlcC4KICAgICIiIgogICAgCiAgICBkZWYgX19pbml0X18oc2VsZiwgZm9yY2VmaWVsZF9uYW1lPSdhbWJlcjE0LWFsbC54bWwnLCBzb2x2ZW50X21vZGVsPU5vbmUpOgogICAgICAgICIiIgogICAgICAgIEluaXRpYWxpemUgdGhlIE1pbmltaXplciB3aXRoIGEgRm9yY2VmaWVsZCBhbmQgU29sdmVudCBNb2RlbC4KICAgICAgICAKICAgICAgICBBcmdzOgogICAgICAgICAgICBmb3JjZWZpZWxkX25hbWU6IFRoZSAicnVsZWJvb2siIGZvciBob3cgYXRvbXMgaW50ZXJhY3QuCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgJ2FtYmVyMTQtYWxsLnhtbCcgZGVzY3JpYmVzIHByb3RlaW4gYXRvbXMgKHBhcmFtZXRlcnMgZm9yIGJvbmQgbGVuZ3RocywKICAgICAgICAgICAgICAgICAgICAgICAgICAgICBhbmdsZXMsIGNoYXJnZXMsIGFuZCBWZFcgcmFkaWkpLgogICAgICAgICAgICBzb2x2ZW50X21vZGVsOiAgIEhvdyB3YXRlciBpcyBzaW11bGF0ZWQuIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICdhcHAuT0JDMicgaXMgYW4gIkltcGxpY2l0IFNvbHZlbnQiIG1vZGVsLiBJbnN0ZWFkIG9mIHNpbXVsYXRpbmcKICAgICAgICAgICAgICAgICAgICAgICAgICAgICB0aG91c2FuZHMgb2YgaW5kaXZpZHVhbCB3YXRlciBtb2xlY3VsZXMgKEV4cGxpY2l0IFNvbHZlbnQpLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgIGl0IHVzZXMgYSBtYXRoZW1hdGljYWwgY29udGludXVtIHRvIGFwcHJveGltYXRlIHdhdGVyJ3MgZGllbGVjdHJpYyAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICBzaGllbGRpbmcgYW5kIGh5ZHJvcGhvYmljIGVmZmVjdHMuIFRoaXMgaXMgbXVjaCBmYXN0ZXIuCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIyMjIE5NUiBOb3RlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgIFNpbmNlIE5NUiBpcyBwZXJmb3JtZWQgaW4gc29sdXRpb24gKG5vdCBjcnlzdGFscyksIGltcGxpY2l0IHNvbHZlbnQgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgYWltcyB0byBhcHByb3hpbWF0ZSB0aGF0IHNvbHV0aW9uIGVudmlyb25tZW50LCBkaXN0aW5jdCBmcm9tIHRoZQogICAgICAgICAgICAgICAgICAgICAgICAgICAgIHZhY3V1bSBvciBjcnlzdGFsIGxhdHRpY2UgYXNzdW1wdGlvbnMgb2Ygb3RoZXIgbWV0aG9kcy4KICAgICAgICAiIiIKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgICMgV2UgYWxsb3cgaW5pdGlhbGl6YXRpb24gdG8gcGFzcyBzbyB0aGUgbW9kdWxlIGNhbiBiZSBpbXBvcnRlZAogICAgICAgICAgICAgIyBidXQgbWV0aG9kcyB3aWxsIGNoZWNrIEhBU19PUEVOTU0KICAgICAgICAgICAgIHBhc3MKCiAgICAgICAgIyBTZXQgZGVmYXVsdCBzb2x2ZW50IG1vZGVsCiAgICAgICAgaWYgc29sdmVudF9tb2RlbCBpcyBOb25lIGFuZCBIQVNfT1BFTk1NOgogICAgICAgICAgICBzb2x2ZW50X21vZGVsID0gYXBwLk9CQzIKCiAgICAgICAgc2VsZi5mb3JjZWZpZWxkX25hbWUgPSBmb3JjZWZpZWxkX25hbWUKICAgICAgICBzZWxmLndhdGVyX21vZGVsID0gJ2FtYmVyMTQvdGlwM3BmYi54bWwnIAogICAgICAgIHNlbGYuc29sdmVudF9tb2RlbCA9IHNvbHZlbnRfbW9kZWwKICAgICAgICAKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgcmV0dXJuCgogICAgICAgICMgTWFwIHNvbHZlbnQgbW9kZWxzIHRvIHRoZWlyIHBhcmFtZXRlciBmaWxlcyBpbiBPcGVuTU0KICAgICAgICAjIFRoZXNlIG5lZWQgdG8gYmUgbG9hZGVkIGFsb25nc2lkZSB0aGUgbWFpbiBmb3JjZWZpZWxkCiAgICAgICAgIyBTdGFuZGFyZCBPcGVuTU0gcGF0aHMgb2Z0ZW4gaGF2ZSAnaW1wbGljaXQvJyBhdCB0aGUgcm9vdAogICAgICAgIHNvbHZlbnRfeG1sX21hcCA9IHsKICAgICAgICAgICAgYXBwLk9CQzI6ICdpbXBsaWNpdC9vYmMyLnhtbCcsCiAgICAgICAgICAgIGFwcC5PQkMxOiAnaW1wbGljaXQvb2JjMS54bWwnLAogICAgICAgICAgICBhcHAuR0JuOiAgJ2ltcGxpY2l0L2dibi54bWwnLAogICAgICAgICAgICBhcHAuR0JuMjogJ2ltcGxpY2l0L2dibjIueG1sJywKICAgICAgICAgICAgYXBwLkhDVDogICdpbXBsaWNpdC9oY3QueG1sJywKICAgICAgICB9CgogICAgICAgICMgQnVpbGQgbGlzdCBvZiBYTUwgZmlsZXMgdG8gbG9hZAogICAgICAgIGZmX2ZpbGVzID0gW3NlbGYuZm9yY2VmaWVsZF9uYW1lLCBzZWxmLndhdGVyX21vZGVsXQogICAgICAgIAogICAgICAgIGlmIHNlbGYuc29sdmVudF9tb2RlbCBpbiBzb2x2ZW50X3htbF9tYXA6CiAgICAgICAgICAgIGZmX2ZpbGVzLmFwcGVuZChzb2x2ZW50X3htbF9tYXBbc2VsZi5zb2x2ZW50X21vZGVsXSkKICAgICAgICAKICAgICAgICB0cnk6CiAgICAgICAgICAgICMgVGhlIEZvcmNlRmllbGQgb2JqZWN0IGxvYWRzIHRoZSBkZWZpbml0aW9ucyBvZiBhdG9tIHR5cGVzIGFuZCBwYXJhbWV0ZXJzLgogICAgICAgICAgICBzZWxmLmZvcmNlZmllbGQgPSBhcHAuRm9yY2VGaWVsZCgqZmZfZmlsZXMpCiAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICBsb2dnZXIuZXJyb3IoZiJGYWlsZWQgdG8gbG9hZCBmb3JjZWZpZWxkIHtmZl9maWxlc306IHtlfSIpCiAgICAgICAgICAgIHJhaXNlCgogICAgZGVmIG1pbmltaXplKHNlbGYsIHBkYl9maWxlX3BhdGg6IHN0ciwgb3V0cHV0X3BhdGg6IHN0ciwgbWF4X2l0ZXJhdGlvbnM6IGludCA9IDAsIHRvbGVyYW5jZTogZmxvYXQgPSAxMC4wKSAtPiBib29sOgogICAgICAgICIiIgogICAgICAgIE1pbmltaXplcyB0aGUgZW5lcmd5IG9mIGEgc3RydWN0dXJlIGFscmVhZHkgY29udGFpbmluZyBjb3JyZWN0IGF0b21zIChpbmNsdWRpbmcgSHlkcm9nZW5zKS4KICAgICAgICAKICAgICAgICBBcmdzOgogICAgICAgICAgICBwZGJfZmlsZV9wYXRoOiBJbnB1dCBQREIgcGF0aC4KICAgICAgICAgICAgb3V0cHV0X3BhdGg6IE91dHB1dCBQREIgcGF0aC4KICAgICAgICAgICAgbWF4X2l0ZXJhdGlvbnM6IExpbWl0IHN0ZXBzICgwID0gdW50aWwgY29udmVyZ2VuY2UpLgogICAgICAgICAgICB0b2xlcmFuY2U6IFRhcmdldCBlbmVyZ3kgY29udmVyZ2VuY2UgdGhyZXNob2xkIChrSi9tb2wpLgogICAgICAgICIiIgogICAgICAgIGlmIG5vdCBIQVNfT1BFTk1NOgogICAgICAgICAgICAgbG9nZ2VyLmVycm9yKCJDYW5ub3QgbWluaW1pemU6IE9wZW5NTSBub3QgZm91bmQuIikKICAgICAgICAgICAgIHJldHVybiBGYWxzZQoKICAgICAgICAjIFRoaXMgbWV0aG9kIGFzc3VtZXMgdGhlIGlucHV0IFBEQiBpcyBwZXJmZWN0IChoYXMgSHlkcm9nZW5zLCBjb3JyZWN0IG5hbWVzKS4KICAgICAgICAjIFNlZSAnYWRkX2h5ZHJvZ2Vuc19hbmRfbWluaW1pemUnIGZvciB0aGUgcm9idXN0IHZlcnNpb24gdXNlZCBieSBzeW50aC1wZGIuCiAgICAgICAgcmV0dXJuIHNlbGYuX3J1bl9zaW11bGF0aW9uKHBkYl9maWxlX3BhdGgsIG91dHB1dF9wYXRoLCBtYXhfaXRlcmF0aW9ucywgdG9sZXJhbmNlLCBhZGRfaHlkcm9nZW5zPUZhbHNlKQoKICAgIGRlZiBhZGRfaHlkcm9nZW5zX2FuZF9taW5pbWl6ZShzZWxmLCBwZGJfZmlsZV9wYXRoOiBzdHIsIG91dHB1dF9wYXRoOiBzdHIpIC0+IGJvb2w6CiAgICAgICAgIiIiCiAgICAgICAgUm9idXN0IG1pbmltaXphdGlvbiBwaXBlbGluZTogQWRkcyBIeWRyb2dlbnMgLT4gQ3JlYXRlcy9NaW5pbWl6ZXMgU3lzdGVtIC0+IFNhdmVzIFJlc3VsdC4KICAgICAgICAKICAgICAgICAjIyMgV2h5IEFkZCBIeWRyb2dlbnM/CiAgICAgICAgWC1yYXkgY3J5c3RhbGxvZ3JhcGh5IG9mdGVuIGRvZXNuJ3QgcmVzb2x2ZSBoeWRyb2dlbiBhdG9tcyBiZWNhdXNlIHRoZXkgaGF2ZSB2ZXJ5IGZldyBlbGVjdHJvbnMuCiAgICAgICAgSG93ZXZlciwgTW9sZWN1bGFyIER5bmFtaWNzIGZvcmNlZmllbGRzIChsaWtlIEFtYmVyKSBhcmUgZXhwbGljaXRseSAiQWxsLUF0b20iLiBUaGV5IFJFUVVJUkUKICAgICAgICBoeWRyb2dlbnMgdG8gY2FsY3VsYXRlIGJvbmQgYW5nbGVzIGFuZCBlbGVjdHJvc3RhdGljcyAoaC1ib25kcykgY29ycmVjdGx5LgogICAgICAgIAogICAgICAgICMjIyBOTVIgUGVyc3BlY3RpdmU6CiAgICAgICAgVW5saWtlIFgtcmF5LCBOTVIgcmVsaWVzIGVudGlyZWx5IG9uIHRoZSBtYWduZXRpYyBzcGluIG9mIHByb3RvbnMgKEgxKS4gSHlkcm9nZW5zIGFyZQogICAgICAgIHRoZSAiZXllcyIgb2YgTk1SLiBDb3JyZWN0bHkgcGxhY2luZyB0aGVtIGlzIGNyaXRpY2FsIG5vdCBqdXN0IGZvciBwaHlzaWNzIGJ1dCBmb3IKICAgICAgICBwcmVkaWN0aW5nIE5PRXMgKE51Y2xlYXIgT3ZlcmhhdXNlciBFZmZlY3RzKSB3aGljaCBkZXBlbmQgb24gSC1IIGRpc3RhbmNlcy4KICAgICAgICBXZSB1c2UgYGFwcC5Nb2RlbGxlcmAgdG8gImd1ZXNzIiB0aGUgc3RhbmRhcmQgcG9zaXRpb25zIG9mIGh5ZHJvZ2VucyBhdCBzcGVjaWZpYyBwSCAoNy4wKS4KICAgICAgICAiIiIKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgIGxvZ2dlci5lcnJvcigiQ2Fubm90IGFkZCBoeWRyb2dlbnM6IE9wZW5NTSBub3QgZm91bmQuIikKICAgICAgICAgICAgIHJldHVybiBGYWxzZQogICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIHNlbGYuX3J1bl9zaW11bGF0aW9uKHBkYl9maWxlX3BhdGgsIG91dHB1dF9wYXRoLCBhZGRfaHlkcm9nZW5zPVRydWUpCgogICAgZGVmIF9ydW5fc2ltdWxhdGlvbihzZWxmLCBpbnB1dF9wYXRoLCBvdXRwdXRfcGF0aCwgbWF4X2l0ZXJhdGlvbnM9MCwgdG9sZXJhbmNlPTEwLjAsIGFkZF9oeWRyb2dlbnM9VHJ1ZSk6CiAgICAgICAgIiIiSW50ZXJuYWwgd29ya2VyIGZvciBzZXR0aW5nIHVwIGFuZCBydW5uaW5nIHRoZSBPcGVuTU0gU2ltdWxhdGlvbi4iIiIKICAgICAgICBsb2dnZXIuaW5mbyhmIlByb2Nlc3NpbmcgcGh5c2ljcyBmb3Ige2lucHV0X3BhdGh9Li4uIikKICAgICAgICAKICAgICAgICB0cnk6CiAgICAgICAgICAgICMgMS4gTG9hZCB0aGUgUERCIGZpbGUgc3RydWN0dXJlICh0b3BvbG9neSBhbmQgcG9zaXRpb25zKQogICAgICAgICAgICBwZGIgPSBhcHAuUERCRmlsZShpbnB1dF9wYXRoKQogICAgICAgICAgICAKICAgICAgICAgICAgIyAyLiBQcmVwYXJlIHRoZSBUb3BvbG9neSAoQWRkIEh5ZHJvZ2VucyBpZiByZXF1ZXN0ZWQpCiAgICAgICAgICAgIGlmIGFkZF9oeWRyb2dlbnM6CiAgICAgICAgICAgICAgICBsb2dnZXIuaW5mbygiQWRkaW5nIG1pc3NpbmcgaHlkcm9nZW5zIChwcm90b25hdGlvbiBzdGF0ZSBAIHBIIDcuMCkuLi4iKQogICAgICAgICAgICAgICAgbW9kZWxsZXIgPSBhcHAuTW9kZWxsZXIocGRiLnRvcG9sb2d5LCBwZGIucG9zaXRpb25zKQogICAgICAgICAgICAgICAgbW9kZWxsZXIuYWRkSHlkcm9nZW5zKHNlbGYuZm9yY2VmaWVsZCwgcEg9Ny4wKQogICAgICAgICAgICAgICAgdG9wb2xvZ3kgPSBtb2RlbGxlci50b3BvbG9neQogICAgICAgICAgICAgICAgcG9zaXRpb25zID0gbW9kZWxsZXIucG9zaXRpb25zCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICB0b3BvbG9neSA9IHBkYi50b3BvbG9neQogICAgICAgICAgICAgICAgcG9zaXRpb25zID0gcGRiLnBvc2l0aW9ucwoKICAgICAgICAgICAgIyAzLiBDcmVhdGUgdGhlICdTeXN0ZW0nCiAgICAgICAgICAgICMgVGhlIFN5c3RlbSBvYmplY3QgY29ubmVjdHMgdGhlIFRvcG9sb2d5IChhdG9tcy9ib25kcykgdG8gdGhlIEZvcmNlZmllbGQgKHBoeXNpY3MgcnVsZXMpLgogICAgICAgICAgICAjIEl0IGNhbGN1bGF0ZXMgYWxsIGZvcmNlcyBhY3Rpbmcgb24gZXZlcnkgYXRvbS4KICAgICAgICAgICAgbG9nZ2VyLmRlYnVnKCJDcmVhdGluZyBPcGVuTU0gU3lzdGVtIChhcHBseWluZyBmb3JjZWZpZWxkIHBhcmFtZXRlcnMpLi4uIikKICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgc3lzdGVtID0gc2VsZi5mb3JjZWZpZWxkLmNyZWF0ZVN5c3RlbSgKICAgICAgICAgICAgICAgICAgICB0b3BvbG9neSwKICAgICAgICAgICAgICAgICAgICBub25ib25kZWRNZXRob2Q9YXBwLk5vQ3V0b2ZmLCAjIE5vIGN1dG9mZiBmb3IgdmFjdXVtL2ltcGxpY2l0IChjYWxjdWxhdGVzIEFMTCBwYWlyd2lzZSBpbnRlcmFjdGlvbnMpCiAgICAgICAgICAgICAgICAgICAgY29uc3RyYWludHM9YXBwLkhCb25kcywgICAgICAgIyBDb25zdHJhaW4gSHlkcm9nZW4gYm9uZCBsZW5ndGhzIChhbGxvd3MgbGFyZ2VyIHRpbWUgc3RlcHMgaW4gTUQpCiAgICAgICAgICAgICAgICAgICAgaW1wbGljaXRTb2x2ZW50PXNlbGYuc29sdmVudF9tb2RlbCAjIENvbnRpbnV1bSB3YXRlciBtb2RlbAogICAgICAgICAgICAgICAgKQogICAgICAgICAgICBleGNlcHQgVmFsdWVFcnJvciBhcyB2ZToKICAgICAgICAgICAgICAgICMgRmFsbGJhY2sgbG9naWMgZm9yIHdoZW4gaW1wbGljaXQgc29sdmVudCBmYWlscyAoY29tbW9uIHdpdGggc29tZSBmb3JjZWZpZWxkIGNvbWJvcykKICAgICAgICAgICAgICAgIGlmICJpbXBsaWNpdFNvbHZlbnQiIGluIHN0cih2ZSk6CiAgICAgICAgICAgICAgICAgICAgbG9nZ2VyLndhcm5pbmcoZiJJbXBsaWNpdCBTb2x2ZW50IHBhcmFtZXRlcnMgbm90IGZvdW5kIGZvciB0aGlzIGZvcmNlZmllbGQgY29uZmlndXJhdGlvbi4gVXNpbmcgVmFjdXVtIGVsZWN0cm9zdGF0aWNzIGluc3RlYWQgKHN0YW5kYXJkIGZhbGxiYWNrKS4iKQogICAgICAgICAgICAgICAgICAgIHN5c3RlbSA9IHNlbGYuZm9yY2VmaWVsZC5jcmVhdGVTeXN0ZW0oCiAgICAgICAgICAgICAgICAgICAgICAgIHRvcG9sb2d5LAogICAgICAgICAgICAgICAgICAgICAgICBub25ib25kZWRNZXRob2Q9YXBwLk5vQ3V0b2ZmLAogICAgICAgICAgICAgICAgICAgICAgICBjb25zdHJhaW50cz1hcHAuSEJvbmRzCiAgICAgICAgICAgICAgICAgICAgICAgICMgaW1wbGljaXRTb2x2ZW50IGRlZmF1bHRzIHRvIE5vbmUgKFZhY3V1bSkKICAgICAgICAgICAgICAgICAgICApCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHJhaXNlIHZlCiAgICAgICAgICAgIAogICAgICAgICAgICAjIDQuIENyZWF0ZSB0aGUgSW50ZWdyYXRvcgogICAgICAgICAgICAjIEFuIEludGVncmF0b3IgaXMgdGhlIG1hdGggZW5naW5lIHRoYXQgbW92ZXMgYXRvbXMgYmFzZWQgb24gZm9yY2VzIChGPW1hKS4KICAgICAgICAgICAgIyAnTGFuZ2V2aW5JbnRlZ3JhdG9yJyBzaW11bGF0ZXMgYSBoZWF0IGJhdGggKGZyaWN0aW9uICsgcmFuZG9tIGNvbGxpc2lvbnMpIHRvIG1haW50YWluIHRlbXBlcmF0dXJlLgogICAgICAgICAgICAjCiAgICAgICAgICAgICMgRWR1Y2F0aW9uYWwgTm90ZToKICAgICAgICAgICAgIyBGb3IgcHVyZSBlbmVyZ3kgbWluaW1pemF0aW9uIChmaW5kaW5nIHRoZSBuZWFyZXN0IHZhbGxleSksIHdlIHRlY2huaWNhbGx5IGRvbid0IG5lZWQgYQogICAgICAgICAgICAjIHRoZXJtb3N0YXQgbGlrZSBMYW5nZXZpbiBiZWNhdXNlIHdlIGFyZW4ndCBzaW11bGF0aW5nIHRpbWUtcmVzb2x2ZWQgbW90aW9uIHlldC4KICAgICAgICAgICAgIyBIb3dldmVyLCBPcGVuTU0gcmVxdWlyZXMgYW4gSW50ZWdyYXRvciB0byBkZWZpbmUgdGhlIFNpbXVsYXRpb24gY29udGV4dC4KICAgICAgICAgICAgIyBJZiB3ZSB3ZXJlIHRvIHJ1biAic2ltdWxhdGlvbi5zdGVwKDEwMDApIiwgdGhpcyBpbnRlZ3JhdG9yIHdvdWxkIGdlbmVyYXRlCiAgICAgICAgICAgICMgcmVhbGlzdGljIEJyb3duaWFuIG1vdGlvbiwgc2ltdWxhdGluZyB0aGVybWFsIGZsdWN0dWF0aW9ucy4KICAgICAgICAgICAgaW50ZWdyYXRvciA9IG1tLkxhbmdldmluSW50ZWdyYXRvcigKICAgICAgICAgICAgICAgIDMwMCAqIHVuaXQua2VsdmluLCAgICAgICAjIFRlbXBlcmF0dXJlIChSb29tIHRlbXApCiAgICAgICAgICAgICAgICAxLjAgLyB1bml0LnBpY29zZWNvbmQsICAgIyBGcmljdGlvbiBjb2VmZmljaWVudAogICAgICAgICAgICAgICAgMi4wICogdW5pdC5mZW10b3NlY29uZHMgICMgVGltZSBzdGVwCiAgICAgICAgICAgICkKICAgICAgICAgICAgCiAgICAgICAgICAgICMgNS4gQ3JlYXRlIHRoZSBTaW11bGF0aW9uIGNvbnRleHQKICAgICAgICAgICAgc2ltdWxhdGlvbiA9IGFwcC5TaW11bGF0aW9uKHRvcG9sb2d5LCBzeXN0ZW0sIGludGVncmF0b3IpCiAgICAgICAgICAgIHNpbXVsYXRpb24uY29udGV4dC5zZXRQb3NpdGlvbnMocG9zaXRpb25zKQogICAgICAgICAgICAKICAgICAgICAgICAgIyBSZXBvcnQgRW5lcmd5IEJFRk9SRSBNaW5pbWl6YXRpb24KICAgICAgICAgICAgc3RhdGVfaW5pdGlhbCA9IHNpbXVsYXRpb24uY29udGV4dC5nZXRTdGF0ZShnZXRFbmVyZ3k9VHJ1ZSkKICAgICAgICAgICAgZV9pbml0ID0gc3RhdGVfaW5pdGlhbC5nZXRQb3RlbnRpYWxFbmVyZ3koKS52YWx1ZV9pbl91bml0KHVuaXQua2lsb2pvdWxlc19wZXJfbW9sZSkKICAgICAgICAgICAgbG9nZ2VyLmluZm8oZiJJbml0aWFsIFBvdGVudGlhbCBFbmVyZ3k6IHtlX2luaXQ6LjJmfSBrSi9tb2wiKQogICAgICAgICAgICAKICAgICAgICAgICAgaWYgZV9pbml0ID4gMWU2OgogICAgICAgICAgICAgICAgbG9nZ2VyLmluZm8oIiAgLT4gSGlnaCBpbml0aWFsIGVuZXJneSBkZXRlY3RlZCBkdWUgdG8gc3RlcmljIGNsYXNoZXMuIE1pbmltaXphdGlvbiB3aWxsIHJlc29sdmUgdGhpcy4iKQogICAgICAgICAgICAKICAgICAgICAgICAgIyA2LiBSdW4gRW5lcmd5IE1pbmltaXphdGlvbiAoR3JhZGllbnQgRGVzY2VudCkKICAgICAgICAgICAgbG9nZ2VyLmluZm8oIk1pbmltaXppbmcgZW5lcmd5Li4uIikKICAgICAgICAgICAgc2ltdWxhdGlvbi5taW5pbWl6ZUVuZXJneSgpCiAgICAgICAgICAgIAogICAgICAgICAgICAjIFJlcG9ydCBFbmVyZ3kgQUZURVIgTWluaW1pemF0aW9uCiAgICAgICAgICAgIHN0YXRlX2ZpbmFsID0gc2ltdWxhdGlvbi5jb250ZXh0LmdldFN0YXRlKGdldEVuZXJneT1UcnVlLCBnZXRQb3NpdGlvbnM9VHJ1ZSkKICAgICAgICAgICAgZV9maW5hbCA9IHN0YXRlX2ZpbmFsLmdldFBvdGVudGlhbEVuZXJneSgpLnZhbHVlX2luX3VuaXQodW5pdC5raWxvam91bGVzX3Blcl9tb2xlKQogICAgICAgICAgICBsb2dnZXIuaW5mbyhmIkZpbmFsIFBvdGVudGlhbCBFbmVyZ3k6ICAge2VfZmluYWw6LjJmfSBrSi9tb2wiKQogICAgICAgICAgICAKICAgICAgICAgICAgIyA3LiBTYXZlIFJlc3VsdAogICAgICAgICAgICB3aXRoIG9wZW4ob3V0cHV0X3BhdGgsICd3JykgYXMgZjoKICAgICAgICAgICAgICAgIGFwcC5QREJGaWxlLndyaXRlRmlsZShzaW11bGF0aW9uLnRvcG9sb2d5LCBzdGF0ZV9maW5hbC5nZXRQb3NpdGlvbnMoKSwgZikKICAgICAgICAgICAgICAgIAogICAgICAgICAgICByZXR1cm4gVHJ1ZQoKICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgIGxvZ2dlci5lcnJvcihmIlBoeXNpY3Mgc2ltdWxhdGlvbiBmYWlsZWQ6IHtlfSIpCiAgICAgICAgICAgIGlmICJ0ZW1wbGF0ZSIgaW4gc3RyKGUpLmxvd2VyKCk6CiAgICAgICAgICAgICAgICBsb2dnZXIuZXJyb3IoIkVycm9yIEhpbnQ6IE9wZW5NTSBjb3VsZG4ndCBtYXRjaCByZXNpZHVlcyB0byB0aGUgZm9yY2VmaWVsZC4gVGhpcyB1c3VhbGx5IG1lYW5zIGF0b21zIGFyZSBtaXNzaW5nIG9yIG5hbWVkIGluY29ycmVjdGx5LiIpCiAgICAgICAgICAgIHJldHVybiBGYWxzZQo="
encoded_generator = "..."
encoded_jc = "IiIiClNjYWxhciBDb3VwbGluZyAoSi1jb3VwbGluZykgY2FsY3VsYXRpb25zLgoKRURVQ0FUSU9OQUwgTk9URSAtIEthcnBsdXMgRXF1YXRpb246Cj09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PQpTY2FsYXIgY291cGxpbmdzIChKLWNvdXBsaW5ncykgYXJlIG1lZGlhdGVkIHRocm91Z2ggY2hlbWljYWwgYm9uZHMuClRoZSAzLWJvbmQgY291cGxpbmcgKF4zSikgZGVwZW5kcyBoZWF2aWx5IG9uIHRoZSB0b3JzaW9uIGFuZ2xlIGJldHdlZW4gdGhlIGF0b21zLgoKRm9yIHRoZSBiYWNrYm9uZSBhbWlkZSBwcm90b24gKEhOKSBhbmQgYWxwaGEgcHJvdG9uIChIQSksIHRoZSBjb3VwbGluZyBeM0pfSE5IYQp0ZWxscyB1cyBhYm91dCB0aGUgUGhpIGFuZ2xlLCBhbmQgdGh1cyB0aGUgc2Vjb25kYXJ5IHN0cnVjdHVyZS4KCkZvcm11bGE6CiAgXjNKID0gQSAqIGNvc14yKHRoZXRhKSArIEIgKiBjb3ModGhldGEpICsgQwoKV2hlcmUgdGhldGEgPSBQaGkgLSA2MCBkZWdyZWVzIChwaGFzZSBzaGlmdCkuClR5cGljYWwgdmFsdWVzOgotIEFscGhhIEhlbGl4IChQaGkgfiAtNjApOiB0aGV0YSB+IC0xMjAgLT4gSiBpcyBzbWFsbCAofjQgSHopCi0gQmV0YSBTaGVldCAoUGhpIH4gLTEyMCk6IHRoZXRhIH4gLTE4MCAtPiBKIGlzIGxhcmdlICh+OSBIeikKLSBSYW5kb20gQ29pbDogQXZlcmFnZWQgKH43IEh6KQoKVGhpcyBhbGxvd3MgTk1SIHNwZWN0cm9zY29waXN0cyB0byBkZXRlcm1pbmUgc2Vjb25kYXJ5IHN0cnVjdHVyZSBqdXN0IGJ5IG1lYXN1cmluZyBKLWNvdXBsaW5ncyEKIiIiCgppbXBvcnQgbnVtcHkgYXMgbnAKaW1wb3J0IGJpb3RpdGUuc3RydWN0dXJlIGFzIHN0cnVjCmltcG9ydCBsb2dnaW5nCmZyb20gdHlwaW5nIGltcG9ydCBEaWN0LCBMaXN0LCBUdXBsZQoKbG9nZ2VyID0gbG9nZ2luZy5nZXRMb2dnZXIoX19uYW1lX18pCgojIFZ1aXN0ZXIgJiBCYXggcGFyYW1ldGVycyAoSi4gQW0uIENoZW0uIFNvYy4gMTk5MywgMTE1LCA3NzcyLTc3NzcpCktBUlBMVVNfUEFSQU1TID0gewogICAgJ0EnOiA2LjUxLAogICAgJ0InOiAtMS43NiwKICAgICdDJzogMS42MAp9CgpkZWYgY2FsY3VsYXRlX2huX2hhX2NvdXBsaW5nKHN0cnVjdHVyZTogc3RydWMuQXRvbUFycmF5KSAtPiBEaWN0W3N0ciwgRGljdFtpbnQsIGZsb2F0XV06CiAgICAiIiIKICAgIENhbGN1bGF0ZSAzSl9ITkhhIGNvdXBsaW5nIGNvbnN0YW50cyBmb3IgdGhlIHByb3RlaW4gYmFja2JvbmUuCiAgICAKICAgIEFyZ3M6CiAgICAgICAgc3RydWN0dXJlOiBBdG9tQXJyYXkgY29udGFpbmluZyB0aGUgcHJvdGVpbgogICAgICAgIAogICAgUmV0dXJuczoKICAgICAgICBEaWN0IGtleWVkIGJ5IENoYWluIElEIC0+IFJlc2lkdWUgSUQgLT4gSi1jb3VwbGluZyB2YWx1ZSAoSHopCiAgICAiIiIKICAgIGxvZ2dlci5pbmZvKCJDYWxjdWxhdGluZyAzSl9ITkhhIHNjYWxhciBjb3VwbGluZ3MuLi4iKQogICAgCiAgICBwaGksIHBzaSwgb21lZ2EgPSBzdHJ1Yy5kaWhlZHJhbF9iYWNrYm9uZShzdHJ1Y3R1cmUpCiAgICAKICAgICMgYmlvdGl0ZSByZXR1cm5zIGFuZ2xlcyBmb3IgZWFjaCByZXNpZHVlLgogICAgIyBUaGUgZmlyc3QgcmVzaWR1ZSBoYXMgbm8gUGhpICh1bmRlZmluZWQpLgogICAgIyBUaGUgY29ycmVzcG9uZGluZyByZXNpZHVlcyBhcmUgc3RydWN0dXJlIHJlc2lkdWVzIHRoYXQgaGF2ZSBiYWNrYm9uZSBhdG9tcy4KICAgICMgV2UgbmVlZCB0byBtYXAgdGhlc2UgYmFjayB0byBSZXMgSURzLgogICAgCiAgICByZXNfc3RhcnRzID0gc3RydWMuZ2V0X3Jlc2lkdWVfc3RhcnRzKHN0cnVjdHVyZSkKICAgICMgRmlsdGVyIHRvIG9ubHkgYW1pbm8gYWNpZHM/IFVzdWFsbHkgc2FmZS4KICAgIAogICAgcmVzdWx0cyA9IHt9CiAgICAKICAgICMgSXRlcmF0ZSBvdmVyIHJlc2lkdWVzCiAgICAjIEFuZ2xlcyBhcnJheSBtYXRjaGVzIG51bWJlciBvZiByZXNpZHVlcwogICAgaWYgbGVuKHBoaSkgIT0gbGVuKHJlc19zdGFydHMpOgogICAgICAgIGxvZ2dlci53YXJuaW5nKGYiTWlzbWF0Y2ggaW4gYmFja2JvbmUgYW5nbGVzIGNvdW50ICh7bGVuKHBoaSl9KSB2cyByZXNpZHVlIGNvdW50ICh7bGVuKHJlc19zdGFydHMpfSkuIikKICAgICAgICByZXR1cm4ge30KICAgICAgICAKICAgIGZvciBpLCBzdGFydF9pZHggaW4gZW51bWVyYXRlKHJlc19zdGFydHMpOgogICAgICAgICMgR2V0IHJlc2lkdWUgaW5mbwogICAgICAgIHJlc19hdG9tcyA9IHN0cnVjdHVyZVtzdGFydF9pZHggOiByZXNfc3RhcnRzW2krMV0gaWYgaSsxIDwgbGVuKHJlc19zdGFydHMpIGVsc2UgTm9uZV0KICAgICAgICBjaGFpbl9pZCA9IHJlc19hdG9tcy5jaGFpbl9pZFswXQogICAgICAgIHJlc19pZCA9IHJlc19hdG9tcy5yZXNfaWRbMF0KICAgICAgICByZXNfbmFtZSA9IHJlc19hdG9tcy5yZXNfbmFtZVswXQogICAgICAgIAogICAgICAgIGlmIGNoYWluX2lkIG5vdCBpbiByZXN1bHRzOgogICAgICAgICAgICByZXN1bHRzW2NoYWluX2lkXSA9IHt9CiAgICAgICAgICAgIAogICAgICAgICMgR2V0IFBoaSBhbmdsZSAoaW4gcmFkaWFucykKICAgICAgICBwaGlfcmFkID0gcGhpW2ldCiAgICAgICAgCiAgICAgICAgIyBDaGVjayBmb3IgTmFOICh1bmRlZmluZWQsIGUuZy4gTi10ZXJtaW51cykKICAgICAgICBpZiBucC5pc25hbihwaGlfcmFkKToKICAgICAgICAgICAgIyBObyBjb3VwbGluZyBkZWZpbmVkCiAgICAgICAgICAgIGNvbnRpbnVlCiAgICAgICAgICAgIAogICAgICAgICMgR2x5Y2luZSBoYXMgSEEyL0hBMywgdXN1YWxseSBhdmVyYWdlZCBvciBzcGVjaWZpYy4KICAgICAgICAjIFRoaXMgZXF1YXRpb24gYXNzdW1lcyBzdGFuZGFyZCBILU4tQ2EtSGEgZ2VvbWV0cnkuCiAgICAgICAgIyBTdGFuZGFyZCBLYXJwbHVzOiB0aGV0YSA9IHBoaSAtIDYwIGRlZyAoLSBwaS8zKQogICAgICAgIHRoZXRhID0gcGhpX3JhZCAtIChucC5kZWcycmFkKDYwLjApKQogICAgICAgIAogICAgICAgICMgQ2FsY3VsYXRlIEoKICAgICAgICAjIEogPSBBIGNvc14yKHRoZXRhKSArIEIgY29zKHRoZXRhKSArIEMKICAgICAgICBjb3NfdGhldGEgPSBucC5jb3ModGhldGEpCiAgICAgICAgal92YWwgPSAoS0FSUExVU19QQVJBTVNbJ0EnXSAqIChjb3NfdGhldGEgKiogMikpICsgXAogICAgICAgICAgICAgICAgKEtBUlBMVVNfUEFSQU1TWydCJ10gKiBjb3NfdGhldGEpICsgXAogICAgICAgICAgICAgICAgS0FSUExVU19QQVJBTVNbJ0MnXQogICAgICAgICAgICAgICAgCiAgICAgICAgIyBBZGQgbm9pc2U/IE9wdGlvbmFsLiBSZWFsIG1lYXN1cmVtZW50cyBoYXZlIGVycm9yIH4wLjUgSHouCiAgICAgICAgIyBGb3IgcHVyZSBlZHVjYXRpb24sIGNsZWFuIGN1cnZlcyBhcmUgYmV0dGVyLgogICAgICAgIAogICAgICAgIHJlc3VsdHNbY2hhaW5faWRdW3Jlc19pZF0gPSByb3VuZChqX3ZhbCwgMikKICAgICAgICAKICAgIHJldHVybiByZXN1bHRzCg=="

with open(physics_file, "w") as f:
    f.write(base64.b64decode(encoded_physics).decode("utf-8"))

with open(generator_file, "w") as f:
    f.write(base64.b64decode(encoded_generator).decode("utf-8"))

with open(jc_file, "w") as f:
    f.write(base64.b64decode(encoded_jc).decode("utf-8"))

print("✅ Hotfixes applied: OpenMM optional + B-factors preserved + J-Couplings added.")

# Force reload
importlib.reload(synth_pdb)
try:
    import synth_pdb.physics
    importlib.reload(synth_pdb.physics)
except ImportError:
    pass
try:
    import synth_pdb.generator
    importlib.reload(synth_pdb.generator)
except ImportError:
    pass
try:
    import synth_pdb.j_coupling
    importlib.reload(synth_pdb.j_coupling)
except ImportError:
    pass


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from synth_pdb.generator import generate_pdb_content
from synth_pdb.relaxation import calculate_relaxation_rates, predict_order_parameters
import biotite.structure.io.pdb as pdb
import io

# 1. Generate a Test Protein (Zinc Finger-like Motif)
# Visualization: Beta Hairpin (Sheet-Turn-Sheet) + Alpha Helix
# This provides a rich mix of rigid and flexible regions.
sequence = "VKITVGGTLTVALGGALALALALALALAA"
structure_def = "1-5:beta,6-8:random,9-13:beta,14-16:random,17-29:alpha"

print("🧬 Generating virtual protein (Zinc Finger-like fold)...")
print("   - Optimizing side-chains (Monte Carlo)...")
print("   - Minimizing energy (OpenMM if available)...")

pdb_content = generate_pdb_content(
    sequence_str=sequence, 
    structure=structure_def, 
    optimize_sidechains=True, 
    minimize_energy=True
)
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)

# Pre-calculate Order Parameters (S2) based on structure
s2_map = predict_order_parameters(structure)
res_ids = sorted(list(s2_map.keys()))
s2_values = [s2_map[r] for r in res_ids]

print("✅ Protein Model Ready!")


### 🧬 The Simulated Protein (Zinc Finger Motif)

We have generated a synthetic **Zinc Finger-like** fold to demonstrate contrast between different secondary structures:
*   **Residues 1-5**: Beta Strand 1 (Rigid, Extended)
*   **Residues 6-8**: Turn (Flexible)
*   **Residues 9-13**: Beta Strand 2 (Rigid, Extended)
*   **Residues 14-16**: Loop (Flexible)
*   **Residues 17-29**: Alpha Helix (Rigid, Compact)

Notice how the physics engine (`synth-pdb`) automatically assigns different **Order Parameters ($S^2$)** to these regions. $S^2$ represents spatial restriction: **1.0** is completely rigid, **0.0** is completely disordered.

In [ ]:
# Visualizing the Structure
import py3Dmol
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def pdb_to_bfactor_map(pdb_str):
    """Extract average B-factor per residue."""
    res_b = {}
    counts = {}
    for line in pdb_str.splitlines():
        if line.startswith("ATOM"):
            try:
                res_id = int(line[22:26].strip())
                b = float(line[60:66].strip())
                if res_id not in res_b:
                    res_b[res_id] = 0.0
                    counts[res_id] = 0
                res_b[res_id] += b
                counts[res_id] += 1
            except ValueError:
                pass
    # Average
    for r in res_b:
        if counts[r] > 0:
            res_b[r] /= counts[r]
    return res_b

res_b_map = pdb_to_bfactor_map(pdb_content)
if not res_b_map:
    res_b_map = {i: 50 for i in range(1, 30)}

vals = list(res_b_map.values())
min_val = min(vals)
max_val = max(vals)

print(f"📏 Data Stats: Min={min_val:.1f}, Max={max_val:.1f}, Mean={np.mean(vals):.1f}")

# Force Physics Range Scaling
# Prevents one outlier (e.g. 100) from squashing all 20s to Blue.
vmin = 15.0  # Rigid Limit
vmax = 65.0  # Flexible Limit

# Setup Viewer
view = py3Dmol.view(width=400, height=300)
view.addModel(pdb_content, 'pdb')

# Apply Logic: Python-calculated Colors
try:
    cmap = matplotlib.colormaps['bwr']
except AttributeError:
    cmap = plt.get_cmap('bwr')

# Clip values to range for color lookup
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

view.setStyle({'cartoon': {'color': 'white'}}) # Base

for r, b_val in res_b_map.items():
    rgba = cmap(norm(b_val))
    hex_color = mcolors.to_hex(rgba)
    view.addStyle({'resi': r}, {'cartoon': {'color': hex_color}})

view.zoomTo()
print("\n--- COLOR GUIDE ---")
print("🔵 BLUE (Cool) = RIGID (Alpha Helices / Beta Sheets)")
print("🔴 RED  (Hot)  = FLEXIBLE (Loops / Termini)")
print("-------------------")
view.show()


### 📊 Guide to the Plots

When you run the simulator below, you will see three coupled plots. Here is how to interpret them:

1.  **$R_1$ (Longitudinal Rate - Blue)**: 
    *   *Physics*: Sensitive to fast interaction fluctuations (nanosecond scale).
    *   *Pattern*: Often relatively flat for folded proteins, but may dip in flexible loops.

2.  **$R_2$ (Transverse Rate - Red)**: 
    *   *Physics*: Sensitive to slow motions (global tumbling) and chemical exchange.
    *   *Key Insight*: **$R_2$ scales with protein size**. Large proteins (slow tumbling) have high $R_2$ rates. High $R_2$ means the signal decays fast (broad lines), making large proteins hard to study!
    *   *Flexible Regions*: $R_2$ drops sharply because local flexibility averages out the magnetic interactions.

3.  **Heteronuclear NOE (Green)**: 
    *   *The Rigidity Sensor*: This is the most robust indicator of local structure.
    *   **Value ~ 0.8**: Rigid backbone (Helix/Sheet).
    *   **Value < 0.6**: Flexible loop/terminus.
    *   **Negative Values**: Extremely flexible or unfolded.

---

In [ ]:
# Interactive Relaxation Dynamics Exploration
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import io

# 1. Create the persistent Image widget
# We will update this widget's value, rather than printing to the log.
plot_image = widgets.Image(format='png', width=800, height=1000)

# 2. Create Sliders
field_slider = widgets.IntSlider(min=400, max=1200, step=100, value=600, description='Field (MHz)')
tumbling_slider = widgets.FloatSlider(min=2.0, max=50.0, step=1.0, value=10.0, description='Tumbling (ns)')

# 3. Update Function
def update_relaxation_plot(change=None):
    field_mhz = field_slider.value
    tau_m_ns = tumbling_slider.value
    
    # Stop Matplotlib from trying to be helpful
    plt.ioff()
    
    # Calculate
    rates = calculate_relaxation_rates(
        structure, 
        field_mhz=field_mhz, 
        tau_m_ns=tau_m_ns, 
        s2_map=s2_map
    )
    r1 = [rates[r]['R1'] for r in res_ids]
    r2 = [rates[r]['R2'] for r in res_ids]
    noe = [rates[r]['NOE'] for r in res_ids]
    
    # Plot to Memory
    # Note: We create a new figure each time but NEVER display it directly.
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    
    ax1.plot(res_ids, r1, 'o-', color='blue', label='R1 (Longitudinal)')
    ax1.set_ylabel('$R_1$ ($s^{-1}$)')
    ax1.set_title(f'Relaxation Dynamics @ {field_mhz} MHz, $\\tau_m$={tau_m_ns}ns')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(res_ids, r2, 's-', color='red', label='R2 (Transverse)')
    ax2.set_ylabel('$R_2$ ($s^{-1}$)')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(res_ids, noe, '^-', color='green', label='HetNOE')
    ax3.set_ylabel('NOE Ratio')
    ax3.set_xlabel('Residue Number')
    ax3.grid(True, alpha=0.3)
    
    for r in res_ids:
        if s2_map[r] < 0.75:
            for ax in [ax1, ax2, ax3]:
                ax.axvspan(r-0.5, r+0.5, color='yellow', alpha=0.1)
    
    plt.tight_layout()
    
    # Save to buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig) # Essential: kill the object
    
    # Update Image Widget
    buf.seek(0)
    plot_image.value = buf.read()

# 4. Bind Events
# continuous_update=False prevents lag/stacking from rapid drags
field_slider.continuous_update = False
tumbling_slider.continuous_update = False

field_slider.observe(update_relaxation_plot, names='value')
tumbling_slider.observe(update_relaxation_plot, names='value')

# 5. Initial Render
update_relaxation_plot()

# 6. Layout UI
ui = widgets.VBox([field_slider, tumbling_slider])
display(ui, plot_image)

### 🧪 Try These Experiments

Use the sliders above to change the "experimental" conditions:

1.  **Simulate a Giant Protein Complex**:
    *   Slide **Tumbling (ns)** to **40.0 ns**.
    *   *Observation*: Look at the **$R_2$ (Red)** plot. It shoots up significantly! This is why NMR is limited to smaller proteins (< 50-100 kDa) without special tricks (like TROSY).
    *   *Note*: The NOE profile stays mostly the same—local rigidity hasn't changed, only the global tumbling.

2.  **Go to High Field**:
    *   Slide **Field (MHz)** from **600** to **1200**.
    *   *Observation*: $R_2$ increases. This is due to **Chemical Shift Anisotropy (CSA)** becoming a stronger relaxation mechanism at high magnetic fields.


# 🔬 Advanced Spectroscopy: Measuring Geometry Directly

Relaxation tells us about **Motion**. J-Couplings and Chemical Shifts tell us about **Geometry**.

Here we simulate two classic experiments:
1.  **J-Coupling ($^3J_{HNH\alpha}$)**: Measures the torsion angle ($\phi$) of the backbone.
2.  **HSQC Spectrum**: The 2D "fingerprint" of the protein.

In [ ]:
# Calculate J-Couplings using the Karplus Equation
from synth_pdb.j_coupling import calculate_hn_ha_coupling

# 1. Calculate
j_couplings = calculate_hn_ha_coupling(structure)

# 2. Extract Data
res_nums = []
j_vals = []
colors = []

for r in res_ids:
    if r in j_couplings.get('A', {}):
        val = j_couplings['A'][r]
        res_nums.append(r)
        j_vals.append(val)
        # Color by physics expectation
        if val < 6.0: colors.append('blue') # Helix-like
        elif val > 8.0: colors.append('red') # Sheet-like
        else: colors.append('gray') # Random/Avg

# 3. Plot
plt.figure(figsize=(10, 4))
plt.bar(res_nums, j_vals, color=colors, alpha=0.7, edgecolor='black')
plt.axhline(4.0, color='blue', linestyle='--', alpha=0.5, label='Alpha Helix (~4 Hz)')
plt.axhline(9.0, color='red', linestyle='--', alpha=0.5, label='Beta Sheet (~9 Hz)')
plt.xlabel('Residue Number')
plt.ylabel('$^3J_{HNH\\alpha}$ (Hz)')
plt.title('Backbone Dihedral Probe: Scalar Couplings')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📏 INTERPRETATION:")
print("Smaller couplings (< 6 Hz) indicate Alpha-Helical turns.")
print("Larger couplings (> 8 Hz) indicate Extended/Beta conformations.")

In [ ]:
# Simulate 2D HSQC Spectrum
from synth_pdb.chemical_shifts import predict_chemical_shifts

# 1. Predict Shifts (SPARTA-lite)
shifts = predict_chemical_shifts(structure)

# 2. Collect (H, N) pairs
h_shifts = []
n_shifts = []
labels = []
ss_colors = []

from synth_pdb.structure_utils import get_secondary_structure
ss_types = get_secondary_structure(structure)

for i, r in enumerate(res_ids):
    chain_shifts = shifts.get('A', {})
    res_shifts = chain_shifts.get(r, {})
    
    if 'H' in res_shifts and 'N' in res_shifts:
        h = res_shifts['H']
        n = res_shifts['N']
        h_shifts.append(h)
        n_shifts.append(n)
        labels.append(f"{structure[structure.res_id==r].res_name[0]}{r}")
        
        # Color by SS
        ss = ss_types[i] if i < len(ss_types) else 'coil'
        if ss == 'alpha': ss_colors.append('blue')
        elif ss == 'beta': ss_colors.append('red')
        else: ss_colors.append('gray')

# 3. Plot HSQC
plt.figure(figsize=(8, 8))
plt.scatter(h_shifts, n_shifts, c=ss_colors, s=100, alpha=0.8, edgecolors='black')

# Standard NMR axes orientation (High -> Low)
plt.xlim(10.5, 6.5)  # Proton: 10 down to 6
plt.ylim(135, 100)   # Nitrogen: 135 down to 100

plt.xlabel('$^{1}$H Chemical Shift (ppm)')
plt.ylabel('$^{15}$N Chemical Shift (ppm)')
plt.title('Synthetic 2D $^{1}$H-$^{15}$N HSQC Spectrum')

# Label peaks
for h, n, txt in zip(h_shifts, n_shifts, labels):
    plt.annotate(txt, (h, n), xytext=(3, 3), textcoords='offset points', fontsize=8)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 INTERPRETATION:")
print("Each dot represents one residue's backbone amide group.")
print("Dispersion (spread) indicates a folded structure.")
print("Collapsed peaks in the center would indicate an unfolded/disordered protein.")